In [14]:
import torch
import pandas as pd
from rouge import Rouge
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import plotly.express as px
import matplotlib.pyplot as plt

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings("ignore")

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [18]:
data = pd.read_csv("../Data/hindi_train.csv")

In [19]:
data

,Id,Heading,Summary,Article
0,hindi_2023_train_0,"गला दबाकर हत्या की; बॉडी बोरे में भरी, लोकल मा...",Kerala Minor Girl Rape Case - केरल के एर्नाकुल...,केरल के एर्नाकुलम जिले में 5 साल की बच्ची से र...
1,hindi_2023_train_1,तेलंगाना में 18 की जान गई; जम्मू-कश्मीर में बा...,इस साल मानसून सीजन में कई राज्यों में भारी तबा...,मानसून सीजन में हुई भारी बारिश ने कई राज्यों म...
2,hindi_2023_train_2,"राजस्थान सरकार बनाएगी कर्ज राहत आयोग, कोर्ट के...",चुनावी साल में राजस्थान सरकार किसानों को लुभान...,चुनावी साल में राजस्थान सरकार किसानों को लुभान...
3,hindi_2023_train_3,"3 से 7 अगस्त तक कर सकेंगे अप्लाय, प्राइस बैंड ...",Non-banking lender SBFC Finance's initial publ...,नॉन बैंकिग फाइनेंस कंपनी 'SBFC फाइनेंस लिमिटेड...
4,hindi_2023_train_4,"डाइनिंग टेबल पर कुकर-कड़ाही न सजाएं, चीन के खा...",स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...,स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...
...,...,...,...,...
21220,hindi_2022_11363,"MSP पर प्रधानमंत्री ने कमेटी बनाई, पराली जलाना...","कृषि मंत्री ने कहा, तीनों कृषि कानूनों को वापस...",नई दिल्ली: कृषि मंत्री नरेंद्र सिंह तोमर ने एम...
21221,hindi_2022_11364,"योगी सरकार ने 9 गेस्ट हाउस का नाम नदियों, धार्...",मुंबई में स्टेट गेस्ट हाउस अब यूपी स्टेट गेस्ट...,लखनऊ: उत्तर प्रदेश की योगी आदित्यनाथ सरकार ने ...
21222,hindi_2022_11365,"Coronavirus से कुछ राहत, 7 लाख के नीचे आए एक्ट...",केंद्रीय स्वास्थ्य मंत्रालय की तरफ से शुक्रवार...,नई दिल्ली। कोरोना वायरस को लेकर कुछ राहत की बा...
21223,hindi_2022_11366,"वाराणसी गंगा नदी के किनारे सबसे साफ शहर बना, प...",लोकसभा में इस शहर का प्रतिनिधित्व करने वाले प्...,नई दिल्ली: केंद्रीय शहरी विकास मंत्री हरदीप सि...


In [24]:
# Remove rows with empty or whitespace-only entries in critical columns
data = data.dropna(subset=['Heading', 'Summary', 'Article'])  # Remove rows where any of these columns are NaN
data = data[data['Heading'].str.strip().astype(bool) & data['Summary'].str.strip().astype(bool) & data['Article'].str.strip().astype(bool)]

# Alternatively, to handle multiple text columns at once where any text field being empty should result in a row being dropped
text_columns = ['Heading', 'Summary', 'Article']
data = data.dropna(subset=text_columns)  # Drop rows where any specified columns are NaN
data = data[data[text_columns].apply(lambda x: x.str.strip().astype(bool)).all(axis=1)]

# Display the cleaned dataframe
print('Length of original data:', len(data))
data

Length of original data: 21224


,Id,Heading,Summary,Article
0,hindi_2023_train_0,"गला दबाकर हत्या की; बॉडी बोरे में भरी, लोकल मा...",Kerala Minor Girl Rape Case - केरल के एर्नाकुल...,केरल के एर्नाकुलम जिले में 5 साल की बच्ची से र...
1,hindi_2023_train_1,तेलंगाना में 18 की जान गई; जम्मू-कश्मीर में बा...,इस साल मानसून सीजन में कई राज्यों में भारी तबा...,मानसून सीजन में हुई भारी बारिश ने कई राज्यों म...
2,hindi_2023_train_2,"राजस्थान सरकार बनाएगी कर्ज राहत आयोग, कोर्ट के...",चुनावी साल में राजस्थान सरकार किसानों को लुभान...,चुनावी साल में राजस्थान सरकार किसानों को लुभान...
3,hindi_2023_train_3,"3 से 7 अगस्त तक कर सकेंगे अप्लाय, प्राइस बैंड ...",Non-banking lender SBFC Finance's initial publ...,नॉन बैंकिग फाइनेंस कंपनी 'SBFC फाइनेंस लिमिटेड...
4,hindi_2023_train_4,"डाइनिंग टेबल पर कुकर-कड़ाही न सजाएं, चीन के खा...",स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...,स्वाद खाने की बुनियाद है। लेकिन अगर खाना सुंदर...
...,...,...,...,...
21220,hindi_2022_11363,"MSP पर प्रधानमंत्री ने कमेटी बनाई, पराली जलाना...","कृषि मंत्री ने कहा, तीनों कृषि कानूनों को वापस...",नई दिल्ली: कृषि मंत्री नरेंद्र सिंह तोमर ने एम...
21221,hindi_2022_11364,"योगी सरकार ने 9 गेस्ट हाउस का नाम नदियों, धार्...",मुंबई में स्टेट गेस्ट हाउस अब यूपी स्टेट गेस्ट...,लखनऊ: उत्तर प्रदेश की योगी आदित्यनाथ सरकार ने ...
21222,hindi_2022_11365,"Coronavirus से कुछ राहत, 7 लाख के नीचे आए एक्ट...",केंद्रीय स्वास्थ्य मंत्रालय की तरफ से शुक्रवार...,नई दिल्ली। कोरोना वायरस को लेकर कुछ राहत की बा...
21223,hindi_2022_11366,"वाराणसी गंगा नदी के किनारे सबसे साफ शहर बना, प...",लोकसभा में इस शहर का प्रतिनिधित्व करने वाले प्...,नई दिल्ली: केंद्रीय शहरी विकास मंत्री हरदीप सि...


In [25]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(data, test_size=0.15, random_state=42)

print("Training Set:")
print(train_data.head())
print("\nValidation Set:")
print(val_data.head())

Training Set:
                          Id  \
8714   hindi_2023_train_8714   
19043        hindi_2022_9186   
15715        hindi_2022_5858   
3156   hindi_2023_train_3156   
15583        hindi_2022_5726   

                                                 Heading  \
8714   बाहर निकलते ही मां और बहनों के गले लगकर रोया; ...   
19043  UP: भदोही में नशे में धुत युवक ने महात्मा गांध...   
15715  UP: एकतरफा प्यार में युवक ने की छात्रा की हत्य...   
3156   12वीं में पासिंग परसेंटेज के आधार पर परीक्षा म...   
15583  उत्तर प्रदेश में कोरोना वायरस से 17 और मौतें, ...   

                                                 Summary  \
8714   TV Actress Tunisha Sharma Suicide Case Update ...   
19043  जिसके बाद कांग्रेस कार्यकर्ता एकत्रित होकर गां...   
15715  हत्या के 24 घंटे भीतर आरोपी को गिरफ्तार कर लिय...   
3156   NEET 2024 - No Minimum Marks Needed For NEET. ...   
15583  उत्‍तर प्रदेश में शुक्रवार को कोरोना वायरस संक...   

                                                 Article  
8714   तुनिष

In [26]:
train_text = train_data['Article']
train_summary = train_data['Summary']
val_text = val_data['Article']
val_summary = val_data['Summary']

## csebuetnlp/mT5_m2o_hindi_crossSum

In [27]:

import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cuda" if torch.cuda.is_available() else "cpu"
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

model_name = "csebuetnlp/mT5_m2o_hindi_crossSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

In [30]:
from tqdm import tqdm

def generate_summaries(texts_to_summarize, tokenizer, model, data_name):
    generated_summary = []

    for i, input_text in tqdm(enumerate(texts_to_summarize), total=len(texts_to_summarize), desc="Processing Texts"):
        input_ids = tokenizer(
            [WHITESPACE_HANDLER(input_text)],
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512
        )["input_ids"].to(device)
        output_ids = model.generate(
            input_ids=input_ids,
            max_length=84,
            no_repeat_ngram_size=2,
            num_beams=4
        )[0]
        summary = tokenizer.decode(
            output_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )
        generated_summary.append(summary)
    print('summary generated for ', data_name)
    return generated_summary

In [29]:
# train_generated_summaries = generate_summaries(train_text, tokenizer, model, "Training Data")
val_generated_summaries = generate_summaries(val_text, tokenizer, model, "Validation Data")

Processing Texts: 100%|██████████| 3184/3184 [48:55<00:00,  1.08it/s] 

summary generated for  Validation Data


In [32]:
Summaries_val = pd.DataFrame(list(zip(val_summary, val_generated_summaries)), columns =['original_summary', 'mT5_m2o_summary'])
Summaries_val.to_csv('../Baseline/val_results_mT5_m2o.csv', sep=',', header=True, index=False)
Summaries_val

,original_summary,mT5_m2o_summary
0,उत्तर भारत के हिस्सों में 29-31 दिसंबर से रात ...,भारत की राजधानी दिल्ली समेत देश के कई हिस्सों ...
1,शनिवार की दोपहर करीब एक बजे सैकड़ों लोगों की म...,उत्तर प्रदेश के मुजफ्फरनगर ज़िले में कुछ लोगों...
2,UP Bhadohi Bahubali Ex-MLA Vijay Mishra इस वक्...,प्रयागराज में अतीक अहमद और अशरफ की हत्या के बा...
3,''दिल्ली चलो'' आह्वान के तहत राष्ट्रीय राजधानी...,दिल्ली में नए कृषि क़ानूनों के विरोध में आज कि...
4,उत्तर प्रदेश के मंत्री कपिल देव अग्रवाल के भाई...,उत्तर प्रदेश के कौशल विकास राज्य मंत्री कपिल द...
...,...,...
3179,US Drone Strike Syria Update; अमेरिका ने सीरिय...,सीरिया में अमेरिकी सेना के एक हवाई हमले में इस...
3180,उत्तर प्रदेश के ग्रटर नोएडा में बृहस्पतिवार सु...,दिल्ली से सटे नोएडा में एक युवती के साथ कथित छ...
3181,Coromandel Express collides with goods train i...,समाचार एजेंसी पीटीआई के मुताबिक शनिवार को ओडिश...
3182,मंत्रिमंडल विस्तार के बाद गुरुवार को प्रधानमंत...,प्रधानमंत्री नरेंद्र मोदी के मंत्रिमंडल विस्ता...


In [33]:
rouge = Rouge()
score = rouge.get_scores(Summaries_val['original_summary'], Summaries_val['mT5_m2o_summary'], avg=True)
mT5_m2o_Rouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
mT5_m2o_Rouge

,rouge-1,rouge-2,rouge-l
recall,0.375584,0.153469,0.311370
precision,0.204170,0.077054,0.168845
f-measure,0.256334,0.098725,0.212139


In [15]:
import pandas as pd
from rouge import Rouge
val_results = pd.read_csv('../Baseline/val_results_mT5_m2o.csv')

rouge = Rouge()
score = rouge.get_scores(val_results['original_summary'], val_results['mT5_m2o_summary'], avg=True)
mT5_m2o_Rouge = pd.DataFrame(score).set_index([['recall','precision','f-measure']])
mT5_m2o_Rouge

,rouge-1,rouge-2,rouge-l
recall,0.375584,0.153469,0.311370
precision,0.204170,0.077054,0.168845
f-measure,0.256334,0.098725,0.212139


In [16]:
import plotly.express as px

fig = px.bar(mT5_m2o_Rouge*100, x=mT5_m2o_Rouge.index, y=mT5_m2o_Rouge.columns,
             barmode='group',
             text_auto='.2s',
             labels={
                     "Algo": "Algorithms",
                     "value": "Rouge Score",
                     "variable": "legend",
                     'index': "Metrics"
                 })

fig.update_layout(width=650,
                  height=400,
                  title={
                  'text': "Score",
                  'y':.96,
                  'x':0.49,
                  'xanchor': 'center',
                  'yanchor': 'top'})

fig.show()

In [23]:
from bert_score import score

def calculate_bert_score(predicted_corpus, reference_corpus):
    p, r, f1 = score(predicted_corpus, reference_corpus, lang="de")
    return p.mean().item(), r.mean().item(), f1.mean().item()

bert_score_val = calculate_bert_score(val_results['mT5_m2o_summary'].tolist(), val_results['original_summary'].tolist())
print(f'BERT Scores:')
print(f'Precision in BERT Score: {bert_score_val[0]}')
print(f'Recall in BERT Score: {bert_score_val[1]}')
print(f'F1 Score in BERT Score: {bert_score_val[2]}')

BERT Scores:
Precision in BERT Score: 0.7449911236763
Recall in BERT Score: 0.6880707740783691
F1 Score in BERT Score: 0.7148173451423645


In [13]:
[x for x in val_summary][:5]

['उत्तर भारत के हिस्सों में 29-31 दिसंबर से रात के तापमान में 3-5 डिग्री सेल्सियस तक की गिरावट दर्ज की जा सकती है। कई राज्यों में शीतलहर चल रही है। यह बात सोमवार को भारत मौसम विभाग (आईएमडी) ने कही।',
 'शनिवार की दोपहर करीब एक बजे सैकड़ों लोगों की मौजूदगी में मंत्री सुरेश राणा के पिता ठाकुर रणवीर सिंह का अंतिम संस्कार किया गया।',
 'UP Bhadohi Bahubali Ex-MLA Vijay Mishra इस वक्त विजय मिश्रा, उनका बेटा विष्णु मिश्रा, भतीजा मनीष मिश्रा जेल में बंद हैं। इन सबकी करोड़ों रुपए की संपत्ति जब्त हो चुकी है।',
 "''दिल्ली चलो'' आह्वान के तहत राष्ट्रीय राजधानी जा रहे किसानों के ट्रैक्टर को शुक्रवार को हरियाणा के भिवानी जिले में एक ट्रक ने टक्कर मार दी, जिसमें एक किसान की मौत हो गई और दो अन्य घायल हो गए।",
 'उत्तर प्रदेश के मंत्री कपिल देव अग्रवाल के भाई के खिलाफ प्रधानमंत्री नरेंद्र मोदी और मुख्यमंत्री योगी आदित्यनाथ की तस्वीरें लगाकर मोबाइल फोन का प्रचार करने के मामले में धोखाधड़ी और जालसाजी का मुकदमा दर्ज किया गया है।']

In [41]:
val_generated_summaries[:5]

['भारत की राजधानी दिल्ली समेत देश के कई हिस्सों में शीतलहर चल रही है.',
 'उत्तर प्रदेश के मुजफ्फरनगर ज़िले में कुछ लोगों ने अपने पिता की मौत का दुख जताया है.',
 'प्रयागराज में अतीक अहमद और अशरफ की हत्या के बाद यूपी सरकार ने कई माफियाओं की सूची जारी की है.',
 'दिल्ली में नए कृषि क़ानूनों के विरोध में आज किसानों ने शांतिपूर्ण प्रदर्शन किया.',
 'उत्तर प्रदेश के कौशल विकास राज्य मंत्री कपिल देव अग्रवाल के भाई के खिलाफ एक विज्ञापन एजेंसी के संचालक के ख़िलाफ हजरतगंज में मुकदमा दर्ज किया गया है.']